<table style="border: none" align="left">
   <tr style="border: none">
      <th style="border: none"><font face="verdana" size="5" color="black"><b>Predict outdoor equipment purchase with IBM Watson Machine Learning</b></th>
      <th style="border: none"><img src="https://github.com/pmservice/customer-satisfaction-prediction/blob/master/app/static/images/ml_icon_gray.png?raw=true" alt="Watson Machine Learning icon" height="40" width="40"></th>
   </tr>
   <tr style="border: none">
       <th style="border: none"><img src="https://github.com/pmservice/wml-sample-models/blob/master/spark/product-line-prediction/images/products_graphics.png?raw=true" alt="Icon"> </th>
   </tr>
</table>

This notebook contains steps and code to get data from the IBM Data Science Experience Community, create a predictive model, and start scoring new data. This notebook introduces commands for getting data and for basic data cleaning and exploration, pipeline creation, model training, model persistance to Watson Machine Learning repository, model deployment, and scoring.

Some familiarity with Python is helpful. This notebook uses Python 2.0 and Apache® Spark 2.0.

You will use a publicly available data set, **GoSales Transactions for Naive Bayes Model**, which details anonymous outdoor equipment purchases. Use the details of this data set to predict clients' interests in terms of product line, such as golf accessories, camping equipment, and others.

**Note**: In this notebook we use GOSales data available on [Data Science Experience Community](https://apsportal.ibm.com/exchange-api/v1/entries/8044492073eb964f46597b4be06ff5ea/data?accessKey=9561295fa407698694b1e254d0099600).

## Learning goals

The learning goals of this notebook are:

-  Load a CSV file into an Apache® Spark DataFrame.
-  Explore data.
-  Prepare data for training and evaluation.
-  Create an Apache® Spark machine learning pipeline.
-  Train and evaluate a model.
-  Persist a pipeline and model in Watson Machine Learning repository.
-  Deploy a model for online scoring using Wastson Machine Learning API.
-  Score sample scoring data using the Watson Machine Learning API.
-  Explore and visualize prediction result using the plotly package.


## Contents

This notebook contains the following parts:

1.	[Setup](#setup)
2.	[Load and explore data](#load)
3.	[Create spark ml model](#model)
4.	[Persist model](#persistence)
5.	[Predict locally and visualize](#visualization)
6.	[Deploy and score in a Cloud](#scoring)
7.	[Summary and next steps](#summary)

<a id="setup"></a>
## 1. Setup

Before you use the sample code in this notebook, you must perform the following setup tasks:

-  Create a [Watson Machine Learning Service](https://console.ng.bluemix.net/catalog/services/ibm-watson-machine-learning/) instance (a free plan is offered). 
-  Upload **GoSales Transactions** data as a data asset in IBM Data Science Experience.
-  Make sure that you are using a Spark 2.0 kernel.

<a id="load"></a>
## 2. Load and explore data

In this section you will load the data as an Apache® Spark DataFrame and perform a basic exploration.

Load the data to the Spark DataFrame by using *wget* to upload the data to gpfs and then *read* method. 

**Example**: First, you need to install required packages. You can do it by running the following code. Run it only one time.<BR><BR>
!pip install wget --user <BR>

In [1]:
#!pip install wget --user
#!pip install totalvoice

In [2]:
import wget

link_to_data = 'https://apsportal.ibm.com/exchange-api/v1/entries/8044492073eb964f46597b4be06ff5ea/data?accessKey=9561295fa407698694b1e254d0099600'
filename = "GoSales_Tx_NaiveBayes.csv"

#filename = wget.download(link_to_data)

The csv file GoSales_Tx_NaiveBayes.csv is availble on gpfs now. Load the file to Apache® Spark DataFrame using code below.

In [3]:
spark = SparkSession.builder.getOrCreate()

df_data = spark.read\
  .format('org.apache.spark.sql.execution.datasources.csv.CSVFileFormat')\
  .option('header', 'true')\
  .option('inferSchema', 'true')\
  .load(filename).cache()

In [4]:
from pyspark.sql import SparkSession
from pyspark.sql import Row
warehouse_location = 'hdfs://hire4.fyre.ibm.com:8020/apps/hive/warehouse'

spark = SparkSession \
    .builder \
    .config("spark.sql.warehouse.dir", warehouse_location) \
    .enableHiveSupport() \
    .getOrCreate()
    
df_data.registerTempTable("temptable")
spark.sql("CREATE TABLE IF NOT EXISTS GOSALES as select * from temptable")

DataFrame[]

In [5]:
df_data = spark.sql("SELECT * FROM GOSALES").cache()

Explore the loaded data by using the following Apache® Spark DataFrame methods:
-  print schema
-  print top ten records
-  count all records

In [6]:
df_data.printSchema()

root
 |-- PRODUCT_LINE: string (nullable = true)
 |-- GENDER: string (nullable = true)
 |-- AGE: integer (nullable = true)
 |-- MARITAL_STATUS: string (nullable = true)
 |-- PROFESSION: string (nullable = true)



As you can see, the data contains five fields. PRODUCT_LINE field is the one we would like to predict (label).

In [7]:
df_data.show()

+--------------------+------+---+--------------+------------+
|        PRODUCT_LINE|GENDER|AGE|MARITAL_STATUS|  PROFESSION|
+--------------------+------+---+--------------+------------+
|Personal Accessories|     M| 27|        Single|Professional|
|Personal Accessories|     F| 39|       Married|       Other|
|Mountaineering Eq...|     F| 39|       Married|       Other|
|Personal Accessories|     F| 56|   Unspecified| Hospitality|
|      Golf Equipment|     M| 45|       Married|     Retired|
|      Golf Equipment|     M| 45|       Married|     Retired|
|   Camping Equipment|     F| 39|       Married|       Other|
|   Camping Equipment|     F| 49|       Married|       Other|
|  Outdoor Protection|     F| 49|       Married|       Other|
|      Golf Equipment|     M| 47|       Married|     Retired|
|      Golf Equipment|     M| 47|       Married|     Retired|
|Mountaineering Eq...|     M| 21|        Single|      Retail|
|Personal Accessories|     F| 66|       Married|       Other|
|   Camp

In [8]:
print "Number of records: " + str(df_data.count())

Number of records: 60252


As you can see, the data set contains 60252 records.

<a id="model"></a>
## 3. Create an Apache® Spark machine learning model

In this section you will learn how to prepare data, create an Apache® Spark machine learning pipeline, and train a model.

### 3.1: Prepare data

In this subsection you will split your data into: train, test and predict datasets.

In [9]:
splitted_data = df_data.randomSplit([0.8, 0.18, 0.02], 24)
train_data = splitted_data[0]
test_data = splitted_data[1]
predict_data = splitted_data[2]

print "Number of training records: " + str(train_data.count())
print "Number of testing records : " + str(test_data.count())
print "Number of prediction records : " + str(predict_data.count())

Number of training records: 48143
Number of testing records : 10937
Number of prediction records : 1172


In [10]:
train_data.printSchema()

root
 |-- PRODUCT_LINE: string (nullable = true)
 |-- GENDER: string (nullable = true)
 |-- AGE: integer (nullable = true)
 |-- MARITAL_STATUS: string (nullable = true)
 |-- PROFESSION: string (nullable = true)



As you can see our data has been successfully split into three datasets: 

-  The train data set, which is the largest group, is used for training.
-  The test data set will be used for model evaluation and is used to test the assumptions of the model.
-  The predict data set will be used for prediction.

### 3.2: Create pipeline and train a model

In this section you will create an Apache® Spark machine learning pipeline and then train the model.

In the first step you need to import the Apache® Spark machine learning packages that will be needed in the subsequent steps.

In [11]:
from pyspark.ml.feature import OneHotEncoder, StringIndexer, IndexToString, VectorAssembler
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml import Pipeline, Model

In the following step, convert all the string fields to numeric ones by using the StringIndexer transformer.

In [12]:
stringIndexer_label = StringIndexer(inputCol="PRODUCT_LINE", outputCol="label").fit(df_data)
stringIndexer_prof = StringIndexer(inputCol="PROFESSION", outputCol="PROFESSION_IX")
stringIndexer_gend = StringIndexer(inputCol="GENDER", outputCol="GENDER_IX")
stringIndexer_mar = StringIndexer(inputCol="MARITAL_STATUS", outputCol="MARITAL_STATUS_IX")

In the following step, create a feature vector by combining all features together.

In [13]:
vectorAssembler_features = VectorAssembler(inputCols=["GENDER_IX", "AGE", "MARITAL_STATUS_IX", "PROFESSION_IX"], outputCol="features")

Next, define estimators you want to use for classification. Random Forest is used in the following example.

In [14]:
rf = RandomForestClassifier(labelCol="label", featuresCol="features")

Finally, indexed labels back to original labels.

In [15]:
labelConverter = IndexToString(inputCol="prediction", outputCol="predictedLabel", labels=stringIndexer_label.labels)

Let's build the pipeline now. A pipeline consists of transformers and an estimator.

In [16]:
pipeline_rf = Pipeline(stages=[stringIndexer_label, stringIndexer_prof, stringIndexer_gend, stringIndexer_mar, vectorAssembler_features, rf, labelConverter])

Now, you can train your Random Forest model by using the previously defined **pipeline** and **train data**.

In [17]:
train_data.printSchema()

root
 |-- PRODUCT_LINE: string (nullable = true)
 |-- GENDER: string (nullable = true)
 |-- AGE: integer (nullable = true)
 |-- MARITAL_STATUS: string (nullable = true)
 |-- PROFESSION: string (nullable = true)



In [18]:
model_rf = pipeline_rf.fit(train_data)

In [19]:
model_rf

PipelineModel_4d718ce71a3bdc1a3d13

You can check your **model accuracy** now. To evaluate the model, use **test data**.

In [20]:
predictions = model_rf.transform(test_data)
evaluatorRF = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluatorRF.evaluate(predictions)

print("Accuracy = %g" % accuracy)
print("Test Error = %g" % (1.0 - accuracy))

Accuracy = 0.585627
Test Error = 0.414373


You can tune your model now to achieve better accuracy. For simplicity of this example tuning section is omitted.

<a id="visualization"></a>
## 4. Predict locally and visualize

In this section you will learn how to score test data using loaded model and visualize the prediction results with plotly package.

### 4.1: Make local prediction using previously loaded model and test data

In this subsection you will score *predict_data* data set.

In [21]:
predictions = model_rf.transform(predict_data)

In [22]:
predict_data.printSchema()

root
 |-- PRODUCT_LINE: string (nullable = true)
 |-- GENDER: string (nullable = true)
 |-- AGE: integer (nullable = true)
 |-- MARITAL_STATUS: string (nullable = true)
 |-- PROFESSION: string (nullable = true)



Preview the results by calling the *show()* method on the predictions DataFrame.

In [23]:
# predictions.select("product_line","gender","age","marital_status","Profession","predictedLabel").show(5)
# .where("product_line <> 'Camping Equipment'")

By tabulating a count, you can see which product line is the most popular.

In [24]:
predictions.select("predictedLabel").groupBy("predictedLabel").count().show(truncate=False)

+------------------------+-----+
|predictedLabel          |count|
+------------------------+-----+
|Camping Equipment       |723  |
|Golf Equipment          |86   |
|Mountaineering Equipment|52   |
|Personal Accessories    |311  |
+------------------------+-----+



### 4.2: Sample visualization of data with Plotly package

In this subsection you will explore prediction results with Plotly, which is an online analytics and data visualization tool.

**Example**: First, you need to install required packages. You can do it by running the following code. Run it only one time.<BR><BR>
!pip install plotly --user <BR>
!pip install cufflinks==0.8.2 --user

In [25]:
#!pip install plotly --user 
#!pip install cufflinks==0.8.2 --user

Import Plotly and other required packages.

In [26]:
import sys
import pandas
import plotly.plotly as py
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import cufflinks as cf
import plotly.graph_objs as go

init_notebook_mode(connected=True)
sys.path.append("".join([os.environ["HOME"]])) 

Convert the Apache Spark DataFrame to a Pandas DataFrame.

In [27]:
predictions_pdf = predictions.select("prediction", "predictedLabel", "GENDER", "AGE", "PROFESSION", "MARITAL_STATUS").toPandas()

Plot a pie chart that shows the predicted product-line interest.

In [28]:
cumulative_stats = predictions_pdf.groupby(['predictedLabel']).count()
product_data = [go.Pie(labels=cumulative_stats.index, values=cumulative_stats['GENDER'])]
product_layout = go.Layout(title='Predicted product line client interest distribution')

fig = go.Figure(data=product_data, layout=product_layout)
iplot(fig)

With this data set, you might want to do some analysis of the mean AGE per product line by using a bar chart.

In [29]:
age_data = [go.Bar(y=predictions_pdf.groupby(['predictedLabel']).mean()["AGE"], x=cumulative_stats.index)]

age_layout = go.Layout(
    title='Mean AGE per predicted product line',
    xaxis=dict(title = "Product Line", showline=False),
    yaxis=dict(title = "Mean AGE"))

fig = go.Figure(data=age_data, layout=age_layout)
iplot(fig)

Based on bar plot you created, you might make the following conclusion: The mean age for clients that are interested in golf equipment is predicted to be over 50 years old.

In [30]:
import ibmos2spark

# @hidden_cell
credentials = {
    'auth_url': 'https://identity.open.softlayer.com',
    'project_id': '37ef207335594d13ab057421343b750e',
    'region': 'dallas',
    'user_id': 'dc930845fc434a17858ff427f7063ee5',
    'username': 'member_5134c5713cf6702e2c107d636228cbb953e916c8',
    'password': 'nn,mx)SFgojI~76E'
}

configuration_name = 'os_aa816606_configs'
bmos = ibmos2spark.bluemix(sc, credentials, configuration_name)

from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

In [31]:
df_data_1 = spark.read\
  .format('org.apache.spark.sql.execution.datasources.csv.CSVFileFormat')\
  .option('header', 'true')\
  .option("inferSchema", "true")\
  .load(bmos.url('WatsonMachineLearning', 'meetup.csv')).cache()

In [32]:
#df_data_1.select("id","sentiment","product_line","age","gender","marital_status","profession","nome_completo","telefone").toPandas()

In [33]:
df_data_1.select("id","sentiment","product_line","age","gender","marital_status","profession").toPandas()

,id,sentiment,product_line,age,gender,marital_status,profession
0,10546,positive,Golf Equipment,27,F,Married,Professional
1,13852,positive,Camping Equipment,17,F,Single,Student
2,15288,negative,Mountaineering Equipment,20,M,Single,Retail
3,25736,positive,Outdoor Protection,23,M,Single,Executive
4,20376,neutral,Personal Accessories,19,M,Single,Sales


In [34]:
# spark.sql("drop table tweet_sales")
df_data_1.registerTempTable("temptable2")
spark.sql("CREATE TABLE IF NOT EXISTS tweet_sales as select * from temptable2")

DataFrame[]

In [35]:
df_data_1 = spark.sql("select * from tweet_sales").cache()

In [36]:
from pyspark.sql.functions import *

In [37]:
df_data_1 = df_data_1.withColumn('PRODUCT_LINE', rtrim(df_data_1.PRODUCT_LINE))\
                     .withColumn('GENDER', rtrim(df_data_1.GENDER))\
                     .withColumn('MARITAL_STATUS', rtrim(df_data_1.MARITAL_STATUS))\
                     .withColumn('PROFESSION', rtrim(df_data_1.PROFESSION))

In [38]:
predictions2 = model_rf.transform(df_data_1)

In [39]:
predictions2.select("id","sentiment","predictedlabel").toPandas()

,id,sentiment,predictedlabel
0,10546,positive,Personal Accessories
1,13852,positive,Personal Accessories
2,15288,negative,Camping Equipment
3,25736,positive,Camping Equipment
4,20376,neutral,Camping Equipment


In [40]:
from totalvoice.cliente import Cliente
cliente = Cliente("88ed71219d42e124f9fc24b998b800c0", 'api.totalvoice.com.br')

In [41]:
desconto = "5%"
produto = ""
nome_cliente = ""
cel = ""
for row in predictions2.rdd.collect():
    if row[3].strip() in "positive":
        desconto = "10%"
    elif row[3].strip() in "neutral":
        desconto = "20%"
    elif row[3].strip() in "negative":
        desconto = "40%"
    produto = row[21]
    nome_cliente = row[4]
    cel = str(row[7]).replace(" ", "").replace("(", "").replace(")", "").replace("-", "")[-11:]
    
    mensagem = "Ola, " +nome_cliente+"! Voce acabou de ganhar "+desconto+" de desconto em qualquer produto da nossa linha de "+produto+"."
    print mensagem
    print cel
#     response = cliente.sms.enviar(c#e#l, mensagem)
#     print(response)

Ola, Claudia Borges! Voce acabou de ganhar 10% de desconto em qualquer produto da nossa linha de Personal Accessories.
11993549075
Ola, Patricia Zago! Voce acabou de ganhar 10% de desconto em qualquer produto da nossa linha de Personal Accessories.
11953839783
Ola, Danilo Thiago Lima! Voce acabou de ganhar 40% de desconto em qualquer produto da nossa linha de Camping Equipment.
11982071684
Ola, Carlos Nonaka Bonfante! Voce acabou de ganhar 10% de desconto em qualquer produto da nossa linha de Camping Equipment.
17991519233
Ola, Gustavo Felipe! Voce acabou de ganhar 20% de desconto em qualquer produto da nossa linha de Camping Equipment.
11941532500


In [43]:
predictions2.select("id","nome_completo").toPandas()

,id,nome_completo
0,10546,Claudia Borges
1,13852,Patricia Zago
2,15288,Danilo Thiago Lima
3,25736,Carlos Nonaka Bonfante
4,20376,Gustavo Felipe
